# Projeto Final - Segunda Etapa: Preparação para Modelagem e Treinamento dos Modelos

Chegamos na segunda etapa do projeto. Trouxemos o dataframe final da última etapa e vamos começar o trabalho. Os próximos passos são:

Aplicar escalonamento nas variáveis numéricas
Utilizar One-Hot Encoding para variáveis categóricas
Verificar o desbalanceamento na variável target
Dividir os Dados em Conjuntos
Depois, minha ideia é treinar os seguintes modelos:

Regrssao logistica,
Randon_forest,
Arvore de decisão,
cat_boost,
xg_boost
Nessa etapa, eu pretendo aplicar a validação cruzada e o gridsearch para afiar o treinmento e encontrar os melhores parâmetros para cada modelo. Assim, buscamos atingir o melhor AUC-ROC possível.

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix, RocCurveDisplay

In [60]:
df = pd.read_csv('dataframe_filtrado_projeto_final.csv')

In [61]:
df.head()

,begin_date,is_client,time_of_contract_years,type,paperless_billing,payment_method,monthly_charges,total_charges,gender,senior_citizen,partner,dependents,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,multiple_lines
0,2017-04-01,1,2.84,One year,False,Mailed check,56.95,1889.50,1,0,False,False,0,1,0,1,0,0,0,0
1,2019-10-01,0,0.17,Month-to-month,True,Mailed check,53.85,108.15,1,0,False,False,0,1,1,0,0,0,0,0
2,2016-05-01,1,3.76,One year,False,Bank transfer (automatic),42.30,1840.75,1,0,False,False,0,1,0,1,1,0,0,0
3,2019-09-01,0,0.17,Month-to-month,True,Electronic check,70.70,151.65,0,0,False,False,1,0,0,0,0,0,0,0
4,2019-03-01,0,0.67,Month-to-month,True,Electronic check,99.65,820.50,0,0,False,False,1,0,0,1,0,1,1,1


## Começando a Modelagem

### One-Hot-Encoding

In [62]:
df = pd.get_dummies(df, columns=['type', 'payment_method'], drop_first=True)

print(df.shape)
df.head()

(6784, 23)


,begin_date,is_client,time_of_contract_years,paperless_billing,monthly_charges,total_charges,gender,senior_citizen,partner,dependents,...,device_protection,tech_support,streaming_tv,streaming_movies,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check
0,2017-04-01,1,2.84,False,56.95,1889.50,1,0,False,False,...,1,0,0,0,0,1,0,0,0,1
1,2019-10-01,0,0.17,True,53.85,108.15,1,0,False,False,...,0,0,0,0,0,0,0,0,0,1
2,2016-05-01,1,3.76,False,42.30,1840.75,1,0,False,False,...,1,1,0,0,0,1,0,0,0,0
3,2019-09-01,0,0.17,True,70.70,151.65,0,0,False,False,...,0,0,0,0,0,0,0,0,1,0
4,2019-03-01,0,0.67,True,99.65,820.50,0,0,False,False,...,1,0,1,1,1,0,0,0,1,0


Aplicamos o One-Hot-Encoding para as variáveis categóricas restantes (duas). Agora, vamos separar features e target

### Dividindo os Dados

In [63]:
target = df['is_client']
features = df.drop(columns='is_client')

In [64]:
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.25,
    stratify=target,
    random_state=12345
)

features_train = features_train.drop(columns=['begin_date'])
features_test = features_test.drop(columns=['begin_date'])

print("Tamanho do conjunto de treino:", features_train.shape)
print("Tamanho do conjunto de teste:", features_test.shape)

Tamanho do conjunto de treino: (5088, 21)
Tamanho do conjunto de teste: (1696, 21)


### Aplicando o Escalonamento

In [65]:
numeric_cols = features.select_dtypes(include=['int64', 'float64']).columns.tolist()

features_train_unscaled = features_train.copy()
features_test_unscaled = features_test.copy()

scaler = StandardScaler()

features_train_scaled = features_train.copy()
features_test_scaled = features_test.copy()

features_train_scaled[numeric_cols] = scaler.fit_transform(features_train[numeric_cols])
features_test_scaled[numeric_cols] = scaler.transform(features_test[numeric_cols])

### Superamostragem

In [66]:
features_majority = features_train[target_train == 1]
features_minority = features_train[target_train == 0]
target_majority = target_train[target_train == 1]
target_minority = target_train[target_train == 0]

repeat_factor = int(len(target_majority) / len(target_minority))
features_minority_upsampled = pd.concat([features_minority] * repeat_factor)
target_minority_upsampled = pd.concat([target_minority] * repeat_factor)

features_train_super = pd.concat([features_majority, features_minority_upsampled])
target_train_super = pd.concat([target_majority, target_minority_upsampled])
features_train_super, target_train_super = shuffle(features_train_super, target_train_super, random_state=42)

scaler = StandardScaler()
features_train_super_scaled = scaler.fit_transform(features_train_super)
target_train_super_scaled = target_train_super.copy()

Depois de feita a divisão, temos o escalonamento das variáveis categóricas e uma superamostragem para tratar o desbalanceamento. Criei variáveis preenchidas com todas as versões possíveis dos conjuntos de dados. Assim, temos versões se nenhuma transformação, ou então apenas com escalonamento, apenas com super amostragem e com mabas as transformações juntas. Assim, poderemos fazer diversos testes para procurar a melhor AUC ROC. 

## Treinando os Modelos

Aqui, vamos começar o treinamento de modelos. Minha abordagem é a seguinte: Vamos testar cada modelo com cada uma das variações dos tipos de dados, usando os parâmetros padrões de cada modelo. Pegaremos o melhor resultado e aplicaremos a validação cruzada e o gridsearch pra otimizar o resultado. Então, passaremos para o próximo.

### Árvore de Decisão

In [67]:
tree_model = DecisionTreeClassifier(random_state=12345)

datasets = {
    'OHE apenas': (features_train, features_test, target_train, target_test),
    'Escalonamento apenas': (features_train_scaled, features_test_scaled, target_train, target_test),
    'Superamostragem apenas': (features_train_super, features_test, target_train_super, target_test),
    'Super + Escalonamento': (features_train_super_scaled, features_test_scaled, target_train_super_scaled, target_test),
}

results = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    tree_model.fit(X_train, y_train)
    y_pred = tree_model.predict(X_test)
    y_proba = tree_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    results[name] = {'AUC-ROC': round(auc, 4), 'Accuracy': round(acc, 4)}

import pandas as pd
results_df = pd.DataFrame(results).T
print(results_df)

                        AUC-ROC  Accuracy
OHE apenas               0.7052    0.7571
Escalonamento apenas     0.7054    0.7565
Superamostragem apenas   0.7155    0.7759
Super + Escalonamento    0.6896    0.7111


Esses resultados me parecem promissores. Faz sentido que o melhor resultado aqui tenha sido o sem escalonamento, porque a Árvore de Decisão vai bem sem essa alteração. Vamos aplicar uma validação cruzada e um gridSeach para otimizar nossa árvore ainda mais.

#### Validação Cruzada e GridSearch na Árvore

In [68]:
tree = DecisionTreeClassifier(random_state=12345)

param_grid = {
    'max_depth': [3, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(
    estimator=tree,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(features_train_super, target_train_super)

print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor AUC-ROC (validação cruzada):", round(grid_search.best_score_, 4))

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Melhores parâmetros: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2}
Melhor AUC-ROC (validação cruzada): 0.8909


Esses resultados já são muito bons, mas precisamos que eles sejam mantidos, denytro do possível, no conjunto de teste. Faremos esse teste.

In [69]:
best_tree = DecisionTreeClassifier(
    random_state=12345,
    criterion='entropy',
    max_depth=15,
    min_samples_split=2,
    min_samples_leaf=1
)

best_tree.fit(features_train_super, target_train_super)

y_pred_test = best_tree.predict(features_test)
y_proba_test = best_tree.predict_proba(features_test)[:, 1]

from sklearn.metrics import roc_auc_score, accuracy_score

auc_test = roc_auc_score(target_test, y_proba_test)
acc_test = accuracy_score(target_test, y_pred_test)

print("AUC-ROC (teste):", round(auc_test, 4))
print("Acurácia (teste):", round(acc_test, 4))

AUC-ROC (teste): 0.7516
Acurácia (teste): 0.7695


Aqui, tivemos uma queda dentro do esperado. Temos um leve overfitting, mas isso é normal. O modelo já atingiu um AUC ROC que nos foi pedido. Vamos seguir com os outros treinamentos e ver se conseguimos resultados ainda melhores.

### Floresta Aleatória

In [70]:
rf_model = RandomForestClassifier(random_state=12345)

datasets = {
    'OHE apenas': (features_train, features_test, target_train, target_test),
    'Escalonamento apenas': (features_train_scaled, features_test_scaled, target_train, target_test),
    'Superamostragem apenas': (features_train_super, features_test, target_train_super, target_test),
    'Super + Escalonamento': (features_train_super_scaled, features_test_scaled, target_train_super_scaled, target_test),
}

results_rf = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    y_proba = rf_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    results_rf[name] = {'AUC-ROC': round(auc, 4), 'Accuracy': round(acc, 4)}

results_rf_df = pd.DataFrame(results_rf).T
print(results_rf_df)

                        AUC-ROC  Accuracy
OHE apenas               0.8624    0.8160
Escalonamento apenas     0.8622    0.8166
Superamostragem apenas   0.8628    0.8054
Super + Escalonamento    0.8421    0.7818


Resultados muito bons, ainda mais considerando que usamos os parâmetros padrões da Floresta e não aplicamos validação ainda. Vamos para essa etapa!

#### Validação Cruzada e GridSearch na Floresta

In [71]:
X_train = features_train_super
y_train = target_train_super

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'criterion': ['gini', 'entropy']
}

rf = RandomForestClassifier(random_state=12345)

grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X_train, y_train)

print("Melhores parâmetros:", grid_search_rf.best_params_)
print("Melhor AUC-ROC (validação cruzada):", round(grid_search_rf.best_score_, 4))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Melhores parâmetros: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Melhor AUC-ROC (validação cruzada): 0.964


Um valor muito alto! Vamos verificar quanto ele nos retorna no conjunto de teste

In [72]:
best_rf = RandomForestClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=200,
    random_state=12345
)

best_rf.fit(features_train_super, target_train_super)

y_test_pred = best_rf.predict(features_test)
y_test_proba = best_rf.predict_proba(features_test)[:, 1]

auc_roc_test = roc_auc_score(target_test, y_test_proba)
acc_test = accuracy_score(target_test, y_test_pred)

print("AUC-ROC (teste):", round(auc_roc_test, 4))
print("Acurácia (teste):", round(acc_test, 4))

AUC-ROC (teste): 0.8615
Acurácia (teste): 0.8137


Um resultado muito bom dentro dos parâmetros esperados pelo projeto. Uma nova queda de cerca de 0.1, mas ainda me parece dentro dos limites aceitáveis.

### Regressão Logística

In [73]:
log_model = LogisticRegression(random_state=12345, solver='liblinear')

datasets = {
    'OHE apenas': (features_train, features_test, target_train, target_test),
    'Escalonamento apenas': (features_train_scaled, features_test_scaled, target_train, target_test),
    'Superamostragem apenas': (features_train_super, features_test, target_train_super, target_test),
    'Super + Escalonamento': (features_train_super_scaled, features_test_scaled, target_train_super_scaled, target_test),
}

results = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    log_model.fit(X_train, y_train)
    y_pred = log_model.predict(X_test)
    y_proba = log_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    results[name] = {'AUC-ROC': round(auc, 4), 'Accuracy': round(acc, 4)}

results_df = pd.DataFrame(results).T
print(results_df)

                        AUC-ROC  Accuracy
OHE apenas               0.8508    0.8042
Escalonamento apenas     0.8528    0.8072
Superamostragem apenas   0.8502    0.7860
Super + Escalonamento    0.8495    0.7795


Bons resultados mais uma vez. Vamos passar o próximo passo no conjunto que leva apenas o escalonamento.

#### Gridsearch e validação na Floresta

In [75]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

grid_search = GridSearchCV(
    estimator=log_model,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1
)

grid_search.fit(features_train_scaled, target_train)

print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor AUC-ROC (validação cruzada):", round(grid_search.best_score_, 4))

Melhores parâmetros: {'C': 100, 'penalty': 'l2'}
Melhor AUC-ROC (validação cruzada): 0.8566


Era esperado, mas esses resultados indicam que, muito provavelmente, não usaremos a regressão aqui. Vamos verificar como ela fica no conjunto de teste.

In [76]:
best_log_model = grid_search.best_estimator_

y_pred_test = best_log_model.predict(features_test_scaled)
y_proba_test = best_log_model.predict_proba(features_test_scaled)[:, 1]

auc_test = roc_auc_score(target_test, y_proba_test)
acc_test = accuracy_score(target_test, y_pred_test)

print("AUC-ROC (teste):", round(auc_test, 4))
print("Acurácia (teste):", round(acc_test, 4))

AUC-ROC (teste): 0.8529
Acurácia (teste): 0.8066


É interessante que a regressão manteve o resultado praticamente igual. Ela foi a melhor na generalização dos dados. Por pouco ela não acaba sendo a escolhida até aqui.

### Catboost

In [77]:
catboost_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_state=12345,
    verbose=0
)

datasets = {
    'OHE apenas': (features_train, features_test, target_train, target_test),
    'Escalonamento apenas': (features_train_scaled, features_test_scaled, target_train, target_test),
    'Superamostragem apenas': (features_train_super, features_test, target_train_super, target_test),
    'Super + Escalonamento': (features_train_super_scaled, features_test_scaled, target_train_super_scaled, target_test),
}

results = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    catboost_model.fit(X_train, y_train)
    y_pred = catboost_model.predict(X_test)
    y_proba = catboost_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    results[name] = {'AUC-ROC': round(auc, 4), 'Accuracy': round(acc, 4)}

catboost_results_df = pd.DataFrame(results).T
print(catboost_results_df)

                        AUC-ROC  Accuracy
OHE apenas               0.8859    0.8343
Escalonamento apenas     0.8859    0.8343
Superamostragem apenas   0.8845    0.8149
Super + Escalonamento    0.8525    0.7718


Resultados muito promissores. Vamos ver o impacto dos parâmetros e da validação cruzada.

#### Gridsearch e validação para o Catboost

In [78]:
catboost_model = CatBoostClassifier(verbose=0, random_state=12345)

catboost_params = {
    'depth': [4, 6, 8],
    'learning_rate': [0.03, 0.1],
    'iterations': [200, 300]
}

grid_catboost = GridSearchCV(
    estimator=catboost_model,
    param_grid=catboost_params,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1
)

grid_catboost.fit(features_train_scaled, target_train)

print("Melhores parâmetros:", grid_catboost.best_params_)
print("Melhor AUC-ROC (validação cruzada):", round(grid_catboost.best_score_, 4))

Melhores parâmetros: {'depth': 4, 'iterations': 300, 'learning_rate': 0.1}
Melhor AUC-ROC (validação cruzada): 0.8916


Também não teve uma grande melhora, mas se esse modelo mantiver os resultyados como a regressão manteve, será o melhor

In [80]:
best_catboost = CatBoostClassifier(
    depth=4,
    iterations=300,
    learning_rate=0.1,
    random_state=12345,
    verbose=0
)

best_catboost.fit(features_train_scaled, target_train)

catboost_preds = best_catboost.predict(features_test_scaled)
catboost_proba = best_catboost.predict_proba(features_test_scaled)[:, 1]

catboost_auc = roc_auc_score(target_test, catboost_proba)
catboost_acc = accuracy_score(target_test, catboost_preds)

print('AUC-ROC (teste):', round(catboost_auc, 4))
print('Acurácia (teste):', round(catboost_acc, 4))

AUC-ROC (teste): 0.8924
Acurácia (teste): 0.8426


Tivemos até uma leve melhora! Ainda nos falta um modelo, mas nossa AUC ROC já superou todos os parâmetros esperados do projeto! Vamos treinar o XGBoost e ver o que conseguimos! Até aqui, a Catboost é a vencedora!

### XGBoost

In [82]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=12345)

datasets = {
    'OHE apenas': (features_train, features_test, target_train, target_test),
    'Escalonamento apenas': (features_train_scaled, features_test_scaled, target_train, target_test),
    'Superamostragem apenas': (features_train_super, features_test, target_train_super, target_test),
    'Super + Escalonamento': (features_train_super_scaled, features_test_scaled, target_train_super_scaled, target_test),
}

results = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    y_proba = xgb_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    results[name] = {'AUC-ROC': round(auc, 4), 'Accuracy': round(acc, 4)}

results_df = pd.DataFrame(results).T
print(results_df)

                        AUC-ROC  Accuracy
OHE apenas               0.8809    0.8325
Escalonamento apenas     0.8808    0.8320
Superamostragem apenas   0.8799    0.8190
Super + Escalonamento    0.8225    0.7376


Mais uma vez, bons valores. Vamos usar os dados com OHE apenas.

#### GridSearch e Validação para o XGBoost

In [84]:
"""xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=12345
)

xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.03, 0.1]
}

grid_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=xgb_params,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1
)

grid_xgb.fit(features_train, target_train)

print("Melhores parâmetros:", grid_xgb.best_params_)
print("Melhor AUC-ROC (validação cruzada):", round(grid_xgb.best_score_, 4))"""

Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Melhor AUC-ROC (validação cruzada): 0.8893


In [85]:
"""Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Melhor AUC-ROC (validação cruzada): 0.8893"""

"Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}\nMelhor AUC-ROC (validação cruzada): 0.8893"

Estou deixando o código e os resultados comentados, porque essa célula precisou de algo entre 1 e 2 horas pra rodar. O resultado ficou um pouco abaixo do Catboost. Vamos ver como ele se sai no conjunto de teste. 

In [86]:
best_xgb = XGBClassifier(
    learning_rate=0.1,
    max_depth=4,
    n_estimators=100,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=12345
)

best_xgb.fit(features_train, target_train)

xgb_preds = best_xgb.predict(features_test)
xgb_proba = best_xgb.predict_proba(features_test)[:, 1]

xgb_auc = roc_auc_score(target_test, xgb_proba)
xgb_acc = accuracy_score(target_test, xgb_preds)

print('AUC-ROC (teste):', round(xgb_auc, 4))
print('Acurácia (teste):', round(xgb_acc, 4))

AUC-ROC (teste): 0.8893
Acurácia (teste): 0.839


Empate técnico com o Catboost, mas, consideando a demora muito maior para treinar e o fato de que esse modelo ainda ficou um pouco atrás, não há muitoa dúvida no qual escolher.

## Avaliando Resultados

Sem grandes segredos, os dois modelos baseados em boost tiveram os melhores desempenhos finais, com o CatBooost sendo o melhor deles. De forma geral, todos os modelos tiveram desempenho muito bom e similar, com exceção da árvore de decisão, que ficou um pouco abaixo. Com isso, concluímos a segunda etapa do Projeto! 